In [1]:
pip install findspark

In [2]:
# Import findspark and initialize.
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [21]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
          SELECT
            YEAR(date) AS Year_Sold,
            CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS 4_Bedroom_Avg_Price
          FROM home_sales
          WHERE bedrooms = 4
          GROUP BY YEAR(date)
          ORDER BY YEAR(date) ASC
          """).show()


+---------+-------------------+
|Year_Sold|4_Bedroom_Avg_Price|
+---------+-------------------+
|     2019|        $300,263.70|
|     2020|        $298,353.78|
|     2021|        $301,819.44|
|     2022|        $296,363.88|
+---------+-------------------+



In [19]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
          SELECT
            YEAR(date_built) AS Year_Built,
            CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS 3_Bed_3_Bath_Avg_Price
          FROM home_sales
          WHERE bedrooms = 3 AND bathrooms = 3
          GROUP BY YEAR(date_built)
          ORDER BY YEAR(date_built) ASC
          """).show()


+----------+----------------------+
|Year_Built|3_Bed_3_Bath_Avg_Price|
+----------+----------------------+
|      2010|           $292,859.62|
|      2011|           $291,117.47|
|      2012|           $293,683.19|
|      2013|           $295,962.27|
|      2014|           $290,852.27|
|      2015|           $288,770.30|
|      2016|           $290,555.07|
|      2017|           $292,676.79|
+----------+----------------------+



In [20]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
          SELECT
            YEAR(date_built) AS Year_Built,
            CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Dream_Home_Avg_Price
          FROM home_sales
          WHERE bedrooms = 3
          AND bathrooms = 3
          AND floors = 2
          AND sqft_living >= 2000
          GROUP BY YEAR(date_built)
          ORDER BY YEAR(date_built) ASC
          """).show()


+----------+--------------------+
|Year_Built|Dream_Home_Avg_Price|
+----------+--------------------+
|      2010|         $285,010.22|
|      2011|         $276,553.81|
|      2012|         $307,539.97|
|      2013|         $303,676.79|
|      2014|         $298,264.72|
|      2015|         $297,609.97|
|      2016|         $293,965.10|
|      2017|         $280,317.58|
+----------+--------------------+



In [9]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
          SELECT
            view,
            CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Avg_Price
          FROM home_sales
          GROUP BY view
          HAVING AVG(price) >= 350000
          ORDER BY view DESC
          """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|    Avg_Price|
+----+-------------+
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
|  80|  $991,767.38|
+----+-------------+
only showing top 20 rows

--- 2.4568958282470703 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
          SELECT
            view,
            CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Avg_Price
          FROM home_sales
          GROUP BY view
          HAVING AVG(price) >= 350000
          ORDER BY view DESC
          """).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|    Avg_Price|
+----+-------------+
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
|  80|  $991,767.38|
+----+-------------+
only showing top 20 rows

--- 2.8856077194213867 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [14]:
# 11. Read the formatted parquet data.
p_df_p=spark.read.parquet('home_sales_partitioned')

In [15]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [16]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
          SELECT
            view,
            CONCAT('$', FORMAT_NUMBER(ROUND(AVG(price), 2), 2)) AS Avg_Price
          FROM p_home_sales_p
          GROUP BY view
          HAVING AVG(price) >= 350000
          ORDER BY view DESC
          """).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|    Avg_Price|
+----+-------------+
|  99|$1,061,201.42|
|  98|$1,053,739.33|
|  97|$1,129,040.15|
|  96|$1,017,815.92|
|  95|$1,054,325.60|
|  94|$1,033,536.20|
|  93|$1,026,006.06|
|  92|  $970,402.55|
|  91|$1,137,372.73|
|  90|$1,062,654.16|
|  89|$1,107,839.15|
|  88|$1,031,719.35|
|  87|$1,072,285.20|
|  86|$1,070,444.25|
|  85|$1,056,336.74|
|  84|$1,117,233.13|
|  83|$1,033,965.93|
|  82|$1,063,498.00|
|  81|$1,053,472.79|
|  80|  $991,767.38|
+----+-------------+
only showing top 20 rows

--- 1.0941276550292969 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [18]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False